### 1. combine `mon_features.pkl` & `unmon_features.pkl` into `features_df`

In [1]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '/content/mon_features.pkl'
UNMON_FILE_PATH = '/content/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER1 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'avg_incoming_burst_size',
    'num_outgoing_packets', 'cumul_max',
    'outgoing_first_30', 'avg_incoming_order_first_30'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER1]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
28995                    32.09               0.163669   
28996                    38.62               0.114350   
28997                    34.93               1.331199   
28998                    11.84               0.083521   
28999                     9.62               0.026874   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

### 2. SVM

Ignore ConvergenceWarning

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

Split the dataset into training and testing sets

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Map all points to have mean=0 and std=1

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train.values)
X_test_scale = scaler.transform(X_test.values)
X_scale = scaler.fit_transform(X.values)

Train and test SVM using RBF kernel

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

clf_rbf = SVC(kernel='rbf', C=100, gamma=0.1, class_weight='balanced')
clf_rbf.fit(X_train_scale, y_train)
y_pred_rbf = clf_rbf.predict(X_test_scale)

print("============ Before Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_rbf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rbf))

============ Before Hyperparameter tuning ============
SVM Accuracy: 0.7558620689655172
Classification Report:
               precision    recall  f1-score   support

          -1       0.98      0.73      0.84      2470
           0       0.57      0.64      0.61        42
           1       0.80      0.80      0.80        51
           2       0.31      0.98      0.47        47
           3       0.47      0.84      0.61        51
           4       0.76      0.79      0.77        43
           5       0.75      0.82      0.78        44
           6       0.83      1.00      0.91        53
           7       0.64      0.65      0.65        55
           8       0.64      0.57      0.61        47
           9       0.64      0.62      0.63        45
          10       0.57      0.73      0.64        44
          11       0.58      0.82      0.68        55
          12       0.72      0.92      0.81        52
          13       0.43      0.42      0.42        57
          14       0.61

### 3. Hyperparameter tuning by using Grid Search

In [6]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

search = HalvingGridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train_scale, y_train)
y_pred_tuned = search.predict(X_test_scale)

print("============ After Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))

============ After Hyperparameter tuning ============
SVM Accuracy: 0.7968275862068965
Classification Report:
               precision    recall  f1-score   support

          -1       0.92      0.89      0.90      2470
           0       0.74      0.76      0.75        42
           1       0.87      0.80      0.84        51
           2       0.47      0.91      0.62        47
           3       0.63      0.73      0.67        51
           4       0.73      0.74      0.74        43
           5       0.80      0.89      0.84        44
           6       0.89      0.94      0.92        53
           7       0.60      0.55      0.57        55
           8       0.73      0.74      0.74        47
           9       0.59      0.64      0.62        45
          10       0.66      0.75      0.70        44
          11       0.69      0.75      0.72        55
          12       0.82      0.90      0.86        52
          13       0.65      0.53      0.58        57
          14       0.62 